In [ ]:
from IPython.display import clear_output
import numpy as np

Rotation stuffs

In [403]:
# Returns a matrix which could be used to rotate any vector or dimension=dim
def get_rotational_matrix(dim=4, theta=None, axes=None):


    if theta is None:
        theta = 2 * np.pi * np.random.random()

    theta %= np.pi

    if axes is None:
        axes = np.random.choice(dim, 2, replace=False)

    i, j = axes[0], axes[1]

    rotational_matrix = np.identity(dim)
    rotational_matrix[i][i] = rotational_matrix[j][j] = np.cos(theta)
    rotational_matrix[i][j] = np.sin(theta)
    rotational_matrix[j][i] = -np.sin(theta)

    return rotational_matrix


def get_random_rotational_matrix(dim, count=10):
    id = np.identity(dim)
    for _ in range(count):
        id = id.dot(get_rotational_matrix(dim=dim))
    return id


Creating data

In [599]:
class Regressor:

    def __init__(self, A_DIM, R_DIM):
        self.A_DIM = A_DIM
        self.R_DIM = R_DIM
        self.W = np.array([np.random.random(
            (self.A_DIM, self.R_DIM)), np.random.random((self.R_DIM, self.A_DIM))])

    def validate(self, a, b):
        self.variance, self.max_error, self.accuracy = self.metrics = self.compare_vector(
            a, b)

    def compare_vector(self, a, b):
        c = a - b
        c = np.squeeze(c)
        v = np.var(c)
        m = np.max(c)
        a = np.exp(- v / m) * 100
        return v, m, a

    def dropout_matrix(self, percent, shape):
        x = np.random.choice(2, shape, p=[1 - percent, percent])
        return x

    def fitUtil(self, training_data, training_label, alpha=0.0001, retain=1):

        hidden_layer = (training_data * self.dropout_matrix(retain,
                        training_data.shape[1])).dot(self.W[0])
        predicted_label = (
            hidden_layer * self.dropout_matrix(retain, hidden_layer.shape[1])).dot(self.W[1])

        error = training_label - predicted_label

        dW0 = - \
            training_data.T.dot(
                2 * error.dot(self.W[1].T)) / training_data.size
        dW1 = - hidden_layer.T.dot(2 * error) / training_data.size

        self.W[0] -= dW0 * alpha
        self.W[1] -= dW1 * alpha

    # Applies cross validation
    def fit(self, vecs, vecsy, epochs, **kwargs):

        vecs = np.array(np.split(vecs, 10))
        vecsy = np.array(np.split(vecsy, 10))

        for i in range(10):

            c_v_training_data, c_v_training_label = np.vstack(
                np.delete(vecs, i, axis=0)), np.vstack(np.delete(vecsy, i, axis=0))
            c_v_testing_data, c_v_testing_label = vecs[i], vecsy[i]

            for epoch in range(epochs // 10):
                self.fitUtil(c_v_training_data, c_v_training_label, **kwargs)

            clear_output()
            self.validate(self.predict(c_v_testing_data), c_v_testing_label)
            print("Metrics", i, ":", self.metrics)

    def predict(self, data):
        return data.dot(self.W[0]).dot(self.W[1])

    def __str__(self):
        return '\n'.join(["Error variance " + str(self.error_varience),
                          "Error amplitude " + str(self.max_error),
                          "Accuracy " + str(self.accuracy),
                          "Weights" + str(self.W)])

    def __repr__(self):
        return '\n'.join(["Error variance " + str(self.error_varience),
                          "Error amplitude " + str(self.max_error),
                          "Accuracy " + str(self.accuracy),
                          "Weights" + str(self.W)])


In [ ]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [737]:
def data_gen(A_DIM, R_DIM, NOISE_AIMPLITUDE, DATA_COUNT, rotation_count = 0, noise = False):
    vecs = (np.random.random((DATA_COUNT, R_DIM)) * 2 * NOISE_AIMPLITUDE) - NOISE_AIMPLITUDE
    if noise:
        vecs = np.append(vecs, np.random.random((DATA_COUNT, A_DIM - R_DIM)), axis = 1)
    else:
        vecs = np.append(vecs, np.zeros((DATA_COUNT, A_DIM - R_DIM)), axis = 1)
    r = get_random_rotational_matrix(A_DIM, count=rotation_count)
    vecs = vecs.dot(r)
    return vecs

In [738]:
regressor = Regressor(7, 5)

C:\Users\joseg\AppData\Local\Temp/ipykernel_14656/2225721749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.W = np.array([np.random.random(


In [730]:
vecs = data_gen(7, 5, 1, 1000, 1)

In [733]:
regressor.fit(vecs, vecs, 80000, retain=0.8, alpha = 0.001)
v1 = data_gen(7, 5, 1, 3000, 1)
regressor.compare_vector(regressor.predict(v1), v1)

Metrics 9 : (0.019410256848580477, 0.34164478280950883, 94.47696414991488)


(0.07218623074399796, 0.8002576881671354, 91.37450063985838)

In [734]:
v1 = data_gen(7, 5, 1, 3000, 112)
regressor.compare_vector(regressor.predict(v1), v1)

(0.10884497463688794, 1.5039745821668062, 93.0185219444801)